# Import

In [1]:
import pandas as pd
import numpy as np
import sqlite3

# Neo4J Queries

    - provider_nodes
        - organization_nodes
        - taxonomy_nodes
    - hospital_nodes
        - organization_nodes
        - taxonomy_nodes
    - referral_relationships

In [2]:
db = sqlite3.connect('../data/hopteam.sqlite')

## provider_nodes
    - entity type 1
    - any cbsa
    - referred to an entity type 2 in cbsa 34980
        - with at least 50 transactions and
        - with less than 50 avg wait time

In [ ]:
%%time

query = """
        
        WITH nash_metro AS (
            SELECT *
            FROM zip_cbsa
            WHERE cbsa = '34980'
        )
        SELECT *
        FROM nash_metro
        INNER JOIN profile
        ON nash_metro.zip = profile.provider_business_practice_location_address_postal_code 

"""

pd.read_sql(query, db)

In [ ]:
query = """

    SELECT *
    FROM referral_from AS rf
    INNER JOIN profile AS p
    ON rf.from_npi = p.npi
    WHERE rf.from_npi IN (
        SELECT from_npi
        FROM referral_from
        INNER JOIN referrals
            USING (referral_id)
        INNER JOIN referral_to
            USING (referral_id)
        INNER JOIN profile
            ON referral_to.to_npi = profile.npi
        INNER JOIN zip_cbsa
            ON profile.provider_business_practice_location_address_postal_code = zip_cbsa.zip
        WHERE transaction_count >= 50
        AND average_day_wait < 50
        AND cbsa = '34980'
    )

"""

pd.read_sql(query, db)